In [1]:
import pandas as pd
import numpy as np
import time
import os
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import string 
import helper_functions as hf

In [2]:
cwd=Path.cwd().resolve().parents[0]

In [3]:
BlogTO = pd.read_csv(os.path.join(cwd, "Data", "BlogTO_mentions.csv"))
print(BlogTO.shape)
BlogTO.head()

(47, 7)


,restaurant,category,stars,location,mention_count,latitude,longitude
0,Tiger Sugar,Bubble Tea,4.0,348A Yonge Street,1,43.658036,-79.381941
1,Chick-fil-A,Fast Food,4.5,709 Yonge Street,11,43.669547,-79.386144
2,Phat Kaphrao,Thai,4.5,3016B Bathurst Street,1,43.717842,-79.429744
3,Azkadenya - Mezza Diner,Middle Eastern,4.5,235 Queen Street West,0,43.650249,-79.388288
4,EAT BKK,Thai,4.5,580 Queen Street W,0,NaN,NaN


In [4]:
subway = pd.read_csv(os.path.join(cwd, "Data", "Torornto_Subway_Station_List.csv"))
print(subway.shape)
subway.head()

(74, 5)


,station,line_number,Line_Color,latitude,longitude
0,Finch,1,Yellow,43.782083,-79.416066
1,North York Centre,1,Yellow,43.768595,-79.412732
2,Sheppard-Yonge,1,Yellow,43.761505,-79.410989
3,York Mills,1,Yellow,43.744069,-79.406720
4,Lawrence,1,Yellow,43.726597,-79.402592


In [5]:
radius=1.1 #km
restaurant_dict=dict()
for i in range(0, BlogTO.shape[0]):
    info_dict=dict()
    in_range_index=list()
    distances=hf.distance_func(BlogTO.loc[i,["latitude","longitude"]].tolist(),subway[["latitude","longitude"]].values.tolist())
    in_range_indice=[index for index,value in enumerate(distances) if value < radius]
    in_range_stations=subway.loc[in_range_indice,"station"].tolist()
    info_dict["number_of_stations_({0}km)".format(radius)]=len(in_range_stations)
    info_dict["list_of_stations_({0}km)".format(radius)]='; '.join(in_range_stations)
    restaurant_dict[BlogTO.loc[i,"restaurant"]]=info_dict

In [6]:
restaurant_info_df=pd.DataFrame(restaurant_dict).T  
restaurant_info_df=restaurant_info_df.reset_index().rename(columns={"index":'restaurant'})
final_BlogTO=pd.merge(BlogTO, restaurant_info_df, on = 'restaurant')

In [7]:
final_BlogTO.sort_values(by=[c for c in final_BlogTO.columns if "number_of_stations" in c][0], ascending =False).head(10)

,restaurant,category,stars,location,mention_count,latitude,longitude,number_of_stations_(1.1km),list_of_stations_(1.1km)
14,Avelo,Vegan,4.5,51 St Nicholas Street,1,43.667066,-79.386168,9,Rosedale; Wellesley; College; Queen's Park; Mu...
1,Chick-fil-A,Fast Food,4.5,709 Yonge Street,11,43.669547,-79.386144,8,Rosedale; Wellesley; College; Museum; St. Geor...
27,Hiwa,Chinese,4.0,64 Edward St,1,43.656726,-79.383707,8,Wellesley; College; Queen; King; St. Andrew; O...
0,Tiger Sugar,Bubble Tea,4.0,348A Yonge Street,1,43.658036,-79.381941,7,Wellesley; College; Queen; King; Osgoode; St. ...
3,Azkadenya - Mezza Diner,Middle Eastern,4.5,235 Queen Street West,0,43.650249,-79.388288,7,Queen; King; Union; St. Andrew; Osgoode; St. P...
44,La Diperie,Ice Cream & Frozen Yogurt,3.5,717 Bay St,0,43.659556,-79.384837,7,Wellesley; College; Queen; Osgoode; St. Patric...
34,Gyubee Japanese Grill - Dundas,Japanese,5.0,157 Dundas St W,0,43.655163,-79.385658,7,College; Queen; King; St. Andrew; Osgoode; St....
21,Boat King of Thai Noodles,Thai,3.0,770 Bay Street,1,43.660101,-79.386004,7,Wellesley; College; Queen; Osgoode; St. Patric...
11,Bloom,Cafes,4.0,596 Yonge street,2,43.665505,-79.385056,6,Wellesley; College; Queen's Park; Museum; Bay;...
26,Mascot Brewery Patio,Breweries,4.0,220 KING STREET WEST,0,43.647468,-79.387134,6,Queen; King; Union; St. Andrew; Osgoode; St. P...


In [8]:
final_BlogTO.to_csv(os.path.join(cwd, "Data", "final_BlogTO.csv"),index=False)